In [1]:
import os
project_path = "C:/workspace/Bus Project"
os.chdir(project_path)

In [2]:
from matplotlib import font_manager, rc
font_name = font_manager.FontProperties(fname="C:/Windows/Fonts/malgun.ttf").get_name()
rc('font', family=font_name)

In [5]:
from multiprocessing import Pool
import multiprocessing
import pandas as pd
import numpy as np
import datetime
from tqdm import tqdm
from functools import partial
import folium

import bus.analyzer as anz
import bus.stay as stay

In [6]:
# 데이터 기간 설정
start_date = datetime.datetime(2019, 6, 1)
end_date = datetime.datetime(2019, 8, 29)

# 로딩할 파일 명 리스트 생성
input_path_list = anz.make_input_path(start_date, end_date)

station_usage_df = anz.parallel_load_total_usage_df(input_path_list, core=8)

# 데이터 로드
user_df = anz.load_user_df()
station_df = anz.load_station_df()
cluster_df = anz.load_cluster_df()
cluster_station_df = anz.load_cluster_station_df()

# 정류장 간 이동 데이터 -> 군집간 이동 데이터
cluster_usage_df = stay.create_cluster_usage_df(station_usage_df, cluster_station_df)

In [ ]:
# 체류시간 추출
walk_df = stay.get_walk_df(cluster_usage_df)

In [ ]:
# 결측값 제거
walk_df = walk_df.dropna()

In [ ]:
# 관광객 데이터 추출
tourist_walk_df = stay.fillter_usage_df(walk_df, user_df, tourist=True)

In [ ]:
# 관광객 체류시간 데이터 추출
walk_time_count_df = stay.analyze_walk_time_count(tourist_walk_df)

In [ ]:
# 체류시간 데이터 시각화
x = walk_time_count_df['stay_time'].head(10000)
y = walk_time_count_df['num'].head(10000)
stay.draw_walk_time_count(x, y)

In [ ]:
# position column 추가
tourist_walk_df = stay.set_position_columns(tourist_walk_df, cluster_df)

In [ ]:
import math
    
def set_distance_column(df):
    d = math.pi*2*6371000/360
    
    df['dist_x'] = df['geton_cluster_longitude'] - df['getoff_cluster_longitude']
    df['dist_y'] = df['geton_cluster_latitude'] - df['getoff_cluster_latitude']
    
    df['dist_x'] = df['dist_x'].apply(lambda x : abs(x))
    df['dist_y'] = df['dist_y'].apply(lambda x : abs(x))
    
    df['distance'] = (df['dist_x']*df['dist_x'] + df['dist_y']*df['dist_y'])**0.5*d
    df['distance'] = df['distance'].apply(lambda x : int(x))
    df['distance'] = df['distance']
    return df

tourist_walk_df = set_distance_column(tourist_walk_df)

In [ ]:
def get_walk_distance_df(walk_df):
    walk_df = walk_df['distance'].value_counts()
    walk_df = pd.DataFrame(walk_df)
    walk_df = walk_df.reset_index().rename(columns = {'index':'distance', 'distance':'num'})
    walk_df = walk_df.sort_values('distance')
    seletor = walk_df.columns
    walk_df = walk_df.reset_index()[seletor]
    return walk_df

walk_distance_df = get_walk_distance_df(tourist_walk_df)

In [ ]:
from matplotlib import pyplot as plt
def draw_walk_distance_count(x, y):
    plt.rcParams["figure.figsize"] = (15,4)
    plt.plot(x, y)
    plt.title('체류 거리 별 이용자 수')
    plt.xlabel("체류 거리")
    plt.ylabel("이용자 수")
    plt.show()
    
x = walk_distance_df['distance'][1:]
y = walk_distance_df['num'][1:]
draw_walk_distance_count(x, y)

In [ ]:
y.describe()

In [ ]:
tourist_walk_df['num'] = 1
# tourist_walk_df[['stay_time', 'distance']].groupby(by=['stay_time', 'distance']).count().reset_index()
a = tourist_walk_df[['stay_time', 'distance', 'num']].groupby(by=['stay_time', 'distance']).count().reset_index()


In [ ]:


b = a[a['num']>=3]
print(len(b))
x = b['distance']
y = b['stay_time']
plt.scatter(x, y, s=1)

In [ ]:
a = tourist_walk_df
print(len(a))
b = a[a['distance'] < 10000]
print(len(b))
c = a[a['stay_time'] < 600]
print(len(c))
bc = b[a['stay_time'] < 600]
print(len(bc))

In [ ]:
tourist_walk_df

In [67]:
x1 = 126.553375
y1 = 33.478634
x2 = 126.507000
y2 = 33.500130
dist = ((x1-x2)**2+(y1-y2)**2)**(1/2) *6500000/360
print(dist)
a = anz.load_cluster_df()
selector = list(a.columns)
a['dist1_x'] = (a['cluster_longitude'] - x1)**2
a['dist1_y'] = (a['cluster_latitude'] - y1)**2
a['dist2_x'] = (a['cluster_longitude'] - x2)**2
a['dist2_y'] = (a['cluster_latitude'] - y2)**2

a['dist1'] = (a['dist1_x'] + a['dist1_y'])**(1/2)
a['dist2'] = (a['dist2_x'] + a['dist2_y'])**(1/2)
a['dist'] = a['dist1'] + a['dist2']
a['dist'] = a['dist']*6500000/360
a['dist'] = a['dist'].apply(lambda x : int(x))
selector.append("dist")
a = a[selector]
a = a.sort_values(by="dist")
a[a['dist']<dist*1.2]

922.9053802597195


,cluster_id,cluster_group,cluster_target,cluster_longitude,cluster_latitude,tour_geton_usage,regident_geton_usage,tour_getoff_usage,regident_getoff_usage,total_usage,dist
1,1,제주시,1,126.553375,33.478634,14866.0,732784.0,10087.0,422057.0,1179794.0,922
765,765,제주시,765,126.507000,33.500130,280.0,7024.0,489.0,4954.0,12747.0,922
261,261,제주시,261,126.519750,33.494835,7.0,2353.0,7.0,2328.0,4695.0,923
38,38,제주시,38,126.512910,33.496831,2777.0,120486.0,2491.0,79848.0,205602.0,923
8,8,제주시,8,126.535535,33.486156,2859.0,194799.0,2307.0,126733.0,326698.0,923
...,...,...,...,...,...,...,...,...,...,...,...
379,379,제주시,379,126.518400,33.508900,20.0,645.0,38.0,698.0,1401.0,1094
207,207,제주시,207,126.540265,33.502785,6.0,4236.0,14.0,1976.0,6232.0,1098
792,792,제주시,792,126.512480,33.481010,2.0,1417.0,0.0,609.0,2028.0,1098
226,226,제주시,226,126.535660,33.505215,18.0,1079.0,75.0,3214.0,4386.0,1102


In [52]:
tourist_walk_df

,user_id,getoff_cluster_id,getoff_datetime,geton_cluster_id,geton_datetime,stay_time,geton_cluster_longitude,geton_cluster_latitude,getoff_cluster_longitude,getoff_cluster_latitude,...,dist_y,distance,num,dist1_x,dist1_y,dist2_x,dist2_y,dist1,dist2,dist
0,00015b56a3bbaf374a77b817350da65a46f96a9a682848...,765.0,2019-06-01 09:35:27,1,2019-06-01 11:44:38,129,126.553375,33.478634,126.507000,33.500130,...,0.021496,5683,1,2.066116e-13,1.322314e-13,0.000000,0.000000,5.821022e-07,0.000000,5.821022e-07
1,1f97062f7421827b86f6edb5abc0b135d0b6d78c97a170...,765.0,2019-06-25 09:53:48,1,2019-06-25 11:35:55,102,126.553375,33.478634,126.507000,33.500130,...,0.021496,5683,1,2.066116e-13,1.322314e-13,0.000000,0.000000,5.821022e-07,0.000000,5.821022e-07
2,27ad6c077cec12261e12107b28580afc2f151c565a4f55...,765.0,2019-06-11 16:46:44,1,2019-06-11 17:50:28,63,126.553375,33.478634,126.507000,33.500130,...,0.021496,5683,1,2.066116e-13,1.322314e-13,0.000000,0.000000,5.821022e-07,0.000000,5.821022e-07
3,3341245d000e330ca253d8e4b85f2e56ebc97162508319...,765.0,2019-06-11 16:46:32,1,2019-06-11 17:50:18,63,126.553375,33.478634,126.507000,33.500130,...,0.021496,5683,1,2.066116e-13,1.322314e-13,0.000000,0.000000,5.821022e-07,0.000000,5.821022e-07
4,4255c99ff8782fffd23b62f019f525995a9d7bbca67ad9...,765.0,2019-07-26 08:12:10,1,2019-07-26 09:46:52,94,126.553375,33.478634,126.507000,33.500130,...,0.021496,5683,1,2.066116e-13,1.322314e-13,0.000000,0.000000,5.821022e-07,0.000000,5.821022e-07
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
229555,d32c7480514e47613d7a79b5246b7459a7d4a864b1848b...,1262.0,2019-07-14 10:25:07,1262,2019-07-14 11:26:40,61,126.795360,33.332910,126.795360,33.332910,...,0.000000,0,1,5.855674e-02,2.123548e-02,0.083151,0.027963,2.824752e-01,0.333338,6.158129e-01
229556,d69e523ed334c0eea0e9c9ede0aed984f7c20cfcada7f7...,1245.0,2019-07-03 15:33:54,1227,2019-07-03 16:00:15,26,126.773905,33.350195,126.783320,33.346880,...,0.003315,1109,1,4.863348e-02,1.649658e-02,0.076353,0.023486,2.552059e-01,0.315972,5.711779e-01
229557,e7059e3dd3e77afe9db251343647017b21864924355633...,1316.0,2019-07-02 12:32:25,1243,2019-07-02 12:52:28,20,126.827200,33.326925,126.789945,33.344060,...,0.017135,4559,1,7.498013e-02,2.301562e-02,0.080058,0.024358,3.130427e-01,0.323134,6.361769e-01
229558,ee796a3984a4f844383066e6ffc3ab1a3d55a9519e0e4d...,485.0,2019-07-16 12:20:53,485,2019-07-16 14:00:17,99,126.611035,33.468785,126.611035,33.468785,...,0.000000,0,1,3.324676e-03,9.700280e-05,0.010823,0.000983,5.849511e-02,0.108654,1.671496e-01


In [56]:
a

,user_id,getoff_cluster_id,getoff_datetime,geton_cluster_id,geton_datetime,stay_time,geton_cluster_longitude,geton_cluster_latitude,getoff_cluster_longitude,getoff_cluster_latitude,dist_x,dist_y,distance,num,dist
0,00015b56a3bbaf374a77b817350da65a46f96a9a682848...,765.0,2019-06-01 09:35:27,1,2019-06-01 11:44:38,129,126.553375,33.478634,126.507000,33.500130,0.046375,0.021496,5683,1,37
1,1f97062f7421827b86f6edb5abc0b135d0b6d78c97a170...,765.0,2019-06-25 09:53:48,1,2019-06-25 11:35:55,102,126.553375,33.478634,126.507000,33.500130,0.046375,0.021496,5683,1,37
2,27ad6c077cec12261e12107b28580afc2f151c565a4f55...,765.0,2019-06-11 16:46:44,1,2019-06-11 17:50:28,63,126.553375,33.478634,126.507000,33.500130,0.046375,0.021496,5683,1,37
3,3341245d000e330ca253d8e4b85f2e56ebc97162508319...,765.0,2019-06-11 16:46:32,1,2019-06-11 17:50:18,63,126.553375,33.478634,126.507000,33.500130,0.046375,0.021496,5683,1,37
4,4255c99ff8782fffd23b62f019f525995a9d7bbca67ad9...,765.0,2019-07-26 08:12:10,1,2019-07-26 09:46:52,94,126.553375,33.478634,126.507000,33.500130,0.046375,0.021496,5683,1,37
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
229555,d32c7480514e47613d7a79b5246b7459a7d4a864b1848b...,1262.0,2019-07-14 10:25:07,1262,2019-07-14 11:26:40,61,126.795360,33.332910,126.795360,33.332910,0.000000,0.000000,0,1,40027836
229556,d69e523ed334c0eea0e9c9ede0aed984f7c20cfcada7f7...,1245.0,2019-07-03 15:33:54,1227,2019-07-03 16:00:15,26,126.773905,33.350195,126.783320,33.346880,0.009415,0.003315,1109,1,37126564
229557,e7059e3dd3e77afe9db251343647017b21864924355633...,1316.0,2019-07-02 12:32:25,1243,2019-07-02 12:52:28,20,126.827200,33.326925,126.789945,33.344060,0.037255,0.017135,4559,1,41351501
229558,ee796a3984a4f844383066e6ffc3ab1a3d55a9519e0e4d...,485.0,2019-07-16 12:20:53,485,2019-07-16 14:00:17,99,126.611035,33.468785,126.611035,33.468785,0.000000,0.000000,0,1,10864721


In [55]:

a['dist'] = a['dist'].apply(lambda x : int(x))

<ipython-input-55-7f3e126942ec>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  a['dist'] = a['dist'].apply(lambda x : int(x))
